In [9]:
cur_dir = !pwd
if cur_dir[0] != "/content":
  print("getting back to content dir")
  %cd /content/
else:
  print("already in /content")  
!rm -rf 046211-project
!git clone https://github.com/ynahum/046211-project.git
%cd 046211-project

getting back to content dir
/content
Cloning into '046211-project'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 94 (delta 48), reused 54 (delta 21), pack-reused 4
Unpacking objects: 100% (94/94), done.
/content/046211-project


In [10]:
!pip install learn2learn

In [11]:
import learn2learn as l2l
import torch
import torch.nn as nn

from meta_learner_module import MetaLearner
from scheduler.batch_loss_schedule import BatchLossSchedule
from scheduler.prediction_similarity_schedule import PredictionSimilaritySchedule
from scheduler.random_schedule import RandomSchedule
import copy 
from config import defaults

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [12]:

args = copy.deepcopy(defaults)

#play with arguments and override defaults here
args['dataset'] = "omniglot"
args['train_sample_size'] = 15

print(args)

dataset = args['dataset']
train_sample_size = args['train_sample_size']
n_test_labels = args['n_test_labels']
n_shots = args['n_shots']
per_task_lr = args['per_task_lr']
meta_lr = args['meta_lr']
adaptation_steps = args['adaptation_steps']
meta_batch_size = args['meta_batch_size']
n_epochs = args['n_epochs']


{'dataset': 'omniglot', 'train_sample_size': 15, 'n_test_labels': 5, 'n_shots': 5, 'per_task_lr': 0.01, 'meta_lr': 0.001, 'meta_batch_size': 4, 'adaptation_steps': 5, 'n_epochs': 50}


In [13]:
print("get tasks")
task_sets = l2l.vision.benchmarks.get_tasksets(
    dataset,
    train_samples=train_sample_size,
    train_ways=n_test_labels,
    test_samples=2 * n_shots,
    test_ways=n_test_labels,
    root='~/data')

print("schedule training")
train_schedule = RandomSchedule(task_sets.train)
#train_schedule = PredictionSimilaritySchedule(task_sets.train, shots=n_shots, ways=n_test_labels, similar_first=True)
#train_schedule = BatchLossSchedule(task_sets.train, shots=n_shots, ways=n_test_labels, hardest_first=True)


get tasks
Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
schedule training


In [14]:
print(f"load model (dataset is {dataset})")
if dataset == "mini-imagenet":
    model = l2l.vision.models.MiniImagenetCNN(n_test_labels)
else:
    model = l2l.vision.models.OmniglotCNN(n_test_labels)
print(model)
model.to(device)

loss = nn.CrossEntropyLoss(reduction='mean')

print(f"create meta learner")
meta_learner = MetaLearner(per_task_lr, meta_lr, adaptation_steps, meta_batch_size, model, loss, device)


load model (dataset is omniglot)
OmniglotCNN(
  (base): ConvBase(
    (0): ConvBlock(
      (normalize): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): ConvBlock(
      (normalize): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (2): ConvBlock(
      (normalize): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (3): ConvBlock(
      (normalize): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
  )
  (features): Sequential(
 

In [15]:
print(f"meta learner train")
meta_learner.meta_train(n_epochs, train_schedule)


meta learner train


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:1013: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten/src/ATen/core/TensorBody.h:417.)
  return self._grad


epoch=0/49, loss=1.607, acc=0.439
epoch=1/49, loss=1.329, acc=0.541
epoch=2/49, loss=1.456, acc=0.486
epoch=3/49, loss=1.510, acc=0.412
epoch=4/49, loss=1.357, acc=0.432
epoch=5/49, loss=1.403, acc=0.446
epoch=6/49, loss=1.344, acc=0.520
epoch=7/49, loss=1.081, acc=0.568
epoch=8/49, loss=1.132, acc=0.568
epoch=9/49, loss=1.136, acc=0.655
epoch=10/49, loss=1.000, acc=0.662
epoch=11/49, loss=1.272, acc=0.500
epoch=12/49, loss=1.226, acc=0.595
epoch=13/49, loss=1.056, acc=0.588
epoch=14/49, loss=1.296, acc=0.507
epoch=15/49, loss=0.958, acc=0.649
epoch=16/49, loss=0.994, acc=0.682
epoch=17/49, loss=1.122, acc=0.568
epoch=18/49, loss=0.988, acc=0.628
epoch=19/49, loss=1.051, acc=0.642
epoch=20/49, loss=0.940, acc=0.662
epoch=21/49, loss=0.970, acc=0.628
epoch=22/49, loss=1.066, acc=0.601
epoch=23/49, loss=0.777, acc=0.743
epoch=24/49, loss=0.802, acc=0.716
epoch=25/49, loss=0.826, acc=0.709
epoch=26/49, loss=0.968, acc=0.703
epoch=27/49, loss=0.900, acc=0.642
epoch=28/49, loss=0.875, acc=0

In [16]:
print(f"meta learner test")
meta_learner.meta_test(task_sets.test)


meta learner test
Meta Test Error 0.8029632419347763
Meta Test Accuracy 0.75
